In [1]:
import pandas as pd
import json
import requests
import urllib.request
import os
from datetime import datetime
import cx_Oracle
from IPython.display import display, HTML

In [2]:
estado = 'Procesando'
ancho = 1800
display(HTML(f"<style>.container {{ width:{ancho}px !important; }}</style>"))
df_formulas = pd.read_csv('formulas.csv',delimiter=';')

In [3]:
#Conexion a la base para levantar las formulas
"""host='racexa03adm01.gobiernocba.gov.ar'
port=1521
sid='moremigd'
user='dc'
password='dc'
sid = cx_Oracle.makedsn(host, port, service_name=sid)

connection = cx_Oracle.connect(user, password, sid, encoding="UTF-8")
cursor = connection.cursor()
cursor.execute('select impuesto, cpto_origen, cpto_calculo, cpto_distri, formula_calculo, formula_distri, distribuye, cta_destino from formulas')
result_set = cursor.fetchall()
df_formulas = pd.DataFrame(result_set, columns=[col[0] for col in cursor.description])
# Close the cursor and the database connection
cursor.close()
connection.close()"""

'host=\'racexa03adm01.gobiernocba.gov.ar\'\nport=1521\nsid=\'moremigd\'\nuser=\'dc\'\npassword=\'dc\'\nsid = cx_Oracle.makedsn(host, port, service_name=sid)\n\nconnection = cx_Oracle.connect(user, password, sid, encoding="UTF-8")\ncursor = connection.cursor()\ncursor.execute(\'select impuesto, cpto_origen, cpto_calculo, cpto_distri, formula_calculo, formula_distri, distribuye, cta_destino from formulas\')\nresult_set = cursor.fetchall()\ndf_formulas = pd.DataFrame(result_set, columns=[col[0] for col in cursor.description])\n# Close the cursor and the database connection\ncursor.close()\nconnection.close()'

In [4]:
#Separo formulas de calculo y distribucion
df_formulas_calc = df_formulas.drop(["FORMULA_DISTRI",'CTA_DESTINO'], axis=1)
df_formulas_calc = df_formulas_calc.drop_duplicates(subset=['IMPUESTO', 'CPTO_ORIGEN', 'CPTO_CALCULO', 'CPTO_DISTRI', 'FORMULA_CALCULO', 'DISTRIBUYE'])
df_formulas_distr = df_formulas.drop(["FORMULA_CALCULO"], axis=1)
df_formulas_distr = df_formulas_distr.drop_duplicates(subset=['IMPUESTO', 'CPTO_ORIGEN', 'CPTO_CALCULO', 'CPTO_DISTRI', 'FORMULA_DISTRI', 'CTA_DESTINO', 'DISTRIBUYE'])

In [5]:
#Abro archivo JSON
f = open('AutomotorVarios.json',encoding="utf8")
#f = open('Embarcacion.json',encoding="utf8")
data = json.load(f)
f.close()

In [6]:
#Se normalida para un DF
df=pd.json_normalize(data['boleta'])
datos_json = df
df

,idEventoPago,idBoleta,idPersona,tipoDocumento,nroDocumento,apellidoNombre,codBarra,montoPagado,idRecaudador,tipoPago,medioPago,cantCuotas,fechaVencimiento,moneda,fechaPago,idCuentaContribuyente,idReferencia,listPagos
0,000165824915,693416110751,4842565083,CUIT,27-1234567-8,AUTOMOTORES,000000000000,15000.0,934,CONTADO,CASH,0,2023-09-12,ARS,2023-09-12,0001658444,None,"[{'idObligacion': '0001658434', 'idEncomendant..."


In [7]:
#Solo para visualizacion
datos_json = data
datos_json

{'boleta': {'idEventoPago': '000165824915',
  'idBoleta': '693416110751',
  'idPersona': '4842565083',
  'tipoDocumento': 'CUIT',
  'nroDocumento': '27-1234567-8',
  'apellidoNombre': 'AUTOMOTORES',
  'codBarra': '000000000000',
  'montoPagado': 15000.0,
  'idRecaudador': '934',
  'tipoPago': 'CONTADO',
  'medioPago': 'CASH',
  'cantCuotas': 0,
  'fechaVencimiento': '2023-09-12',
  'moneda': 'ARS',
  'fechaPago': '2023-09-12',
  'idCuentaContribuyente': '0001658444',
  'idReferencia': None,
  'listPagos': [{'idObligacion': '0001658434',
    'idEncomendante': 'AUT-V10',
    'idPago': '000165862502',
    'codBarra': '000000000000',
    'idTipoObligacion': 'AUT-V10',
    'descripcionTipoObligacion': 'AUT-V10',
    'fechaVencimientoObligacion': '2023-07-10',
    'fechaInicioObligacion': '2023-06-01',
    'idExterno': 'AF829DI',
    'montoPagado': 13730.0,
    'idPersona': '4842565083',
    'tipoDocumento': 'CUIT',
    'nroDocumento': '27-24474534-8',
    'apellidoNombre': 'MAINARDI, MARIA 

In [8]:
# Agregar la lista 'listDetallesPago' al JSON para procesar
detalles_pago = datos_json['boleta']['listPagos'][0]['listDetallesPago']
#Extraigo Lista de Pagos
lista_pagos = datos_json['boleta']['listPagos']

In [9]:
detalle_pagos = lista_pagos
detalle_pagos

[{'idObligacion': '0001658434',
  'idEncomendante': 'AUT-V10',
  'idPago': '000165862502',
  'codBarra': '000000000000',
  'idTipoObligacion': 'AUT-V10',
  'descripcionTipoObligacion': 'AUT-V10',
  'fechaVencimientoObligacion': '2023-07-10',
  'fechaInicioObligacion': '2023-06-01',
  'idExterno': 'AF829DI',
  'montoPagado': 13730.0,
  'idPersona': '4842565083',
  'tipoDocumento': 'CUIT',
  'nroDocumento': '27-24474534-8',
  'apellidoNombre': 'MAINARDI, MARIA LORENA',
  'totalJuicio': 0.0,
  'honorariosJudiciales': 0.0,
  'gastosJudiciales': 0.0,
  'listDetallesPago': [{'idTransaccion': '000160299486',
    'tipoMovimiento': 'AUTOASMT',
    'montoPagado': 6110.2,
    'caracteristica': [{'tipo': 'CMCCCDC', 'valor': 'AUTOASMT'}]},
   {'idTransaccion': '000160299486',
    'tipoMovimiento': 'AUTMUN',
    'montoPagado': 6110.2,
    'caracteristica': [{'tipo': 'CMCCCDC', 'valor': 'AUTMUN'}]},
   {'idTransaccion': '000160543020',
    'tipoMovimiento': 'REDONDEO',
    'montoPagado': -7.07,
    '

In [10]:
#Extraigo los valores de los detalles de pago
df_detalles_pago = pd.DataFrame(columns=[
    'idTransaccion',
    'tipoMovimiento',
    'montoPagado',
    # ... otros campos que quieran incluir ...
])

# Acceder a la lista de pagos
lista_pagos = datos_json['boleta']['listPagos']


for pago in lista_pagos:
    detalles_pago = pago['listDetallesPago']
    for detalle in detalles_pago:

        df_detalles_pago = pd.concat([df_detalles_pago, pd.DataFrame(detalle, index=[0])], ignore_index=True)


df_detalles_pago


C:\Users\a446788\AppData\Local\Temp\ipykernel_15368\983502263.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_detalles_pago = pd.concat([df_detalles_pago, pd.DataFrame(detalle, index=[0])], ignore_index=True)


,idTransaccion,tipoMovimiento,montoPagado,caracteristica
0,000160299486,AUTOASMT,6110.20,"{'tipo': 'CMCCCDC', 'valor': 'AUTOASMT'}"
1,000160299486,AUTMUN,6110.20,"{'tipo': 'CMCCCDC', 'valor': 'AUTMUN'}"
2,000160543020,REDONDEO,-7.07,"{'tipo': 'CMCCCDC', 'valor': 'GENERAL'}"
3,000169652350,AUTAFTP,1516.67,"{'tipo': 'CMCCCDC', 'valor': 'AUTFTP'}"
4,000160299486,EMBASMT,150123.00,"{'tipo': 'CMCCCDC', 'valor': 'EMBASMT'}"
5,000160299486,REDONDEO,-123.00,"{'tipo': 'CMCCCDC', 'valor': 'GENERAL'}"


In [11]:
lista_pagos

[{'idObligacion': '0001658434',
  'idEncomendante': 'AUT-V10',
  'idPago': '000165862502',
  'codBarra': '000000000000',
  'idTipoObligacion': 'AUT-V10',
  'descripcionTipoObligacion': 'AUT-V10',
  'fechaVencimientoObligacion': '2023-07-10',
  'fechaInicioObligacion': '2023-06-01',
  'idExterno': 'AF829DI',
  'montoPagado': 13730.0,
  'idPersona': '4842565083',
  'tipoDocumento': 'CUIT',
  'nroDocumento': '27-24474534-8',
  'apellidoNombre': 'MAINARDI, MARIA LORENA',
  'totalJuicio': 0.0,
  'honorariosJudiciales': 0.0,
  'gastosJudiciales': 0.0,
  'listDetallesPago': [{'idTransaccion': '000160299486',
    'tipoMovimiento': 'AUTOASMT',
    'montoPagado': 6110.2,
    'caracteristica': [{'tipo': 'CMCCCDC', 'valor': 'AUTOASMT'}]},
   {'idTransaccion': '000160299486',
    'tipoMovimiento': 'AUTMUN',
    'montoPagado': 6110.2,
    'caracteristica': [{'tipo': 'CMCCCDC', 'valor': 'AUTMUN'}]},
   {'idTransaccion': '000160543020',
    'tipoMovimiento': 'REDONDEO',
    'montoPagado': -7.07,
    '

In [12]:
detalle_pagos = df_detalles_pago
detalle_pagos

,idTransaccion,tipoMovimiento,montoPagado,caracteristica
0,000160299486,AUTOASMT,6110.20,"{'tipo': 'CMCCCDC', 'valor': 'AUTOASMT'}"
1,000160299486,AUTMUN,6110.20,"{'tipo': 'CMCCCDC', 'valor': 'AUTMUN'}"
2,000160543020,REDONDEO,-7.07,"{'tipo': 'CMCCCDC', 'valor': 'GENERAL'}"
3,000169652350,AUTAFTP,1516.67,"{'tipo': 'CMCCCDC', 'valor': 'AUTFTP'}"
4,000160299486,EMBASMT,150123.00,"{'tipo': 'CMCCCDC', 'valor': 'EMBASMT'}"
5,000160299486,REDONDEO,-123.00,"{'tipo': 'CMCCCDC', 'valor': 'GENERAL'}"


In [13]:
#extraigo de la caracteristica el tipo de mov
df_detalles_pago['tipo_mov'] = df_detalles_pago['caracteristica'].apply(lambda x: x['valor'])
detalles_pago = df_detalles_pago.drop("caracteristica", axis=1)
detalles_pago = pd.merge(detalles_pago, df_formulas_calc, left_on='tipo_mov', right_on='CPTO_ORIGEN')
detalles_pago

,idTransaccion,tipoMovimiento,montoPagado,tipo_mov,IMPUESTO,CPTO_ORIGEN,CPTO_CALCULO,CPTO_DISTRI,FORMULA_CALCULO,DISTRIBUYE
0,000160299486,AUTOASMT,6110.20,AUTOASMT,AUT,AUTOASMT,AUTOASMT,AUTOASMT,df.montoPagado + df.montoRecargo,S
1,000160299486,AUTMUN,6110.20,AUTMUN,AUT,AUTMUN,AUTMUN,AUTMUN,df.montoPagado + df.montoRecargo,S
2,000160543020,REDONDEO,-7.07,GENERAL,AUT,GENERAL,GENERAL,GENERAL,df.montoPagado,N
3,000160299486,REDONDEO,-123.00,GENERAL,AUT,GENERAL,GENERAL,GENERAL,df.montoPagado,N
4,000169652350,AUTAFTP,1516.67,AUTFTP,AUT,AUTFTP,AUTFTP,AUTFTP,df.montoPagado,N
5,000160299486,EMBASMT,150123.00,EMBASMT,EMB,EMBASMT,EMBASMT,EMBASMT,df.montoPagado + df.montoRecargo,NaN


In [14]:
#Calculo el monto de recargos y descuentos
indice_general = detalles_pago[detalles_pago['CPTO_CALCULO'] == 'GENERAL'].index[0]
monto_recargo = detalles_pago.loc[indice_general, 'montoPagado']

In [15]:
# Calcula porcentaje de GENERAL (Redondeo) si se distribuye
existe_autmun = 'AUTMUN' in detalles_pago['CPTO_ORIGEN'].values
#existe_general = 'GENERAL' in df_calculo['CPTO_CALCULO'].values
distribuye = 'S' in detalles_pago['DISTRIBUYE'].values

if existe_autmun and distribuye:
    detalles_pago['montoRecargo'] = monto_recargo * .5
else:
    detalles_pago['montoRecargo'] = monto_recargo
df=detalles_pago
detalles_pago

,idTransaccion,tipoMovimiento,montoPagado,tipo_mov,IMPUESTO,CPTO_ORIGEN,CPTO_CALCULO,CPTO_DISTRI,FORMULA_CALCULO,DISTRIBUYE,montoRecargo
0,000160299486,AUTOASMT,6110.20,AUTOASMT,AUT,AUTOASMT,AUTOASMT,AUTOASMT,df.montoPagado + df.montoRecargo,S,-3.535
1,000160299486,AUTMUN,6110.20,AUTMUN,AUT,AUTMUN,AUTMUN,AUTMUN,df.montoPagado + df.montoRecargo,S,-3.535
2,000160543020,REDONDEO,-7.07,GENERAL,AUT,GENERAL,GENERAL,GENERAL,df.montoPagado,N,-3.535
3,000160299486,REDONDEO,-123.00,GENERAL,AUT,GENERAL,GENERAL,GENERAL,df.montoPagado,N,-3.535
4,000169652350,AUTAFTP,1516.67,AUTFTP,AUT,AUTFTP,AUTFTP,AUTFTP,df.montoPagado,N,-3.535
5,000160299486,EMBASMT,150123.00,EMBASMT,EMB,EMBASMT,EMBASMT,EMBASMT,df.montoPagado + df.montoRecargo,NaN,-3.535


In [16]:
#Separo recargos de conceptos validos
df_recargo = detalles_pago[detalles_pago['CPTO_CALCULO'] == 'GENERAL']
df_calculo = detalles_pago[detalles_pago['CPTO_CALCULO'] != 'GENERAL']
df_calculo

,idTransaccion,tipoMovimiento,montoPagado,tipo_mov,IMPUESTO,CPTO_ORIGEN,CPTO_CALCULO,CPTO_DISTRI,FORMULA_CALCULO,DISTRIBUYE,montoRecargo
0,000160299486,AUTOASMT,6110.20,AUTOASMT,AUT,AUTOASMT,AUTOASMT,AUTOASMT,df.montoPagado + df.montoRecargo,S,-3.535
1,000160299486,AUTMUN,6110.20,AUTMUN,AUT,AUTMUN,AUTMUN,AUTMUN,df.montoPagado + df.montoRecargo,S,-3.535
4,000169652350,AUTAFTP,1516.67,AUTFTP,AUT,AUTFTP,AUTFTP,AUTFTP,df.montoPagado,N,-3.535
5,000160299486,EMBASMT,150123.00,EMBASMT,EMB,EMBASMT,EMBASMT,EMBASMT,df.montoPagado + df.montoRecargo,NaN,-3.535


In [17]:
#Aplico formula para calculo de monto final
pd.options.mode.chained_assignment = None
df_list = []
for formula in df_calculo['FORMULA_CALCULO'].unique():
#    print(formula)
    df_new = df_calculo[df_calculo['FORMULA_CALCULO'] == formula].copy()
    df_new['montoCalculado'] = pd.eval(formula)
    df_list.append(df_new)
df_calculo = pd.concat(df_list, ignore_index=True)
df_calculo

,idTransaccion,tipoMovimiento,montoPagado,tipo_mov,IMPUESTO,CPTO_ORIGEN,CPTO_CALCULO,CPTO_DISTRI,FORMULA_CALCULO,DISTRIBUYE,montoRecargo,montoCalculado
0,000160299486,AUTOASMT,6110.20,AUTOASMT,AUT,AUTOASMT,AUTOASMT,AUTOASMT,df.montoPagado + df.montoRecargo,S,-3.535,6106.665
1,000160299486,AUTMUN,6110.20,AUTMUN,AUT,AUTMUN,AUTMUN,AUTMUN,df.montoPagado + df.montoRecargo,S,-3.535,6106.665
2,000160299486,EMBASMT,150123.00,EMBASMT,EMB,EMBASMT,EMBASMT,EMBASMT,df.montoPagado + df.montoRecargo,NaN,-3.535,150119.465
3,000169652350,AUTAFTP,1516.67,AUTFTP,AUT,AUTFTP,AUTFTP,AUTFTP,df.montoPagado,N,-3.535,1516.670


In [18]:
df_calculo = df_calculo.groupby('CPTO_CALCULO')['montoCalculado'].sum().reset_index()
calculado = df_calculo.rename(columns={'CPTO_CALCULO': 'tipoMovimiento', 'montoCalculado': 'montoPagado'})
calculado['montoPagado'] = calculado['montoPagado'].round(2)
calculado

,tipoMovimiento,montoPagado
0,AUTFTP,1516.67
1,AUTMUN,6106.66
2,AUTOASMT,6106.66
3,EMBASMT,150119.46


In [19]:
v_salida = calculado
v_salida['idTransaccion'] = f"{datetime.now().year % 100}{datetime.now().month:02}{datetime.now().day:02}{datetime.now().hour:02}{datetime.now().minute:02}{datetime.now().second:02}{datetime.now().microsecond // 10000:02}"
v_salida['caracteristica'] = '[{''tipo'': ''CMCCCDC'', ''valor'':'' }]'
v_salida

,tipoMovimiento,montoPagado,idTransaccion,caracteristica
0,AUTFTP,1516.67,23110221372263,"[{tipo: CMCCCDC, valor: }]"
1,AUTMUN,6106.66,23110221372263,"[{tipo: CMCCCDC, valor: }]"
2,AUTOASMT,6106.66,23110221372263,"[{tipo: CMCCCDC, valor: }]"
3,EMBASMT,150119.46,23110221372263,"[{tipo: CMCCCDC, valor: }]"


In [20]:
#Elimino lista de Pagos
if 'listDetallesPago' in datos_json['boleta']['listPagos'][0]:
    del datos_json['boleta']['listPagos'][0]['listDetallesPago']
df_as_list = v_salida.to_dict(orient='records')
datos_json['boleta']['listPagos'][0]['listDetallesPago'] = df_as_list
#datos_json

In [21]:
with open("outputAUT.json", "w") as outfile:
#with open("outputEMB.json", "w") as outfile:
    json.dump(datos_json, outfile, indent=4)

In [22]:
distri = pd.merge(calculado, df_formulas_distr, left_on='tipoMovimiento', right_on='CPTO_DISTRI')
distri = pd.DataFrame(distri)
df=distri
df

,tipoMovimiento,montoPagado,idTransaccion,caracteristica,IMPUESTO,CPTO_ORIGEN,CPTO_CALCULO,CPTO_DISTRI,FORMULA_DISTRI,DISTRIBUYE,CTA_DESTINO
0,AUTFTP,1516.67,23110221372263,"[{tipo: CMCCCDC, valor: }]",AUT,AUTFTP,AUTFTP,AUTFTP,df.montoPagado,N,900-0000251229/08
1,AUTMUN,6106.66,23110221372263,"[{tipo: CMCCCDC, valor: }]",AUT,AUTMUN,AUTMUN,AUTMUN,df.montoPagado*.95,S,900-0000013093/02
2,AUTMUN,6106.66,23110221372263,"[{tipo: CMCCCDC, valor: }]",AUT,AUTMUN,AUTMUN,AUTMUN,df.montoPagado*.05,S,900-0000251229/08
3,AUTOASMT,6106.66,23110221372263,"[{tipo: CMCCCDC, valor: }]",AUT,AUTOASMT,AUTOASMT,AUTOASMT,df.montoPagado,S,900-0000000201/03
4,EMBASMT,150119.46,23110221372263,"[{tipo: CMCCCDC, valor: }]",EMB,EMBASMT,EMBASMT,EMBASMT,df.montoPagado,NaN,900-0000000201/03


In [23]:
pd.options.mode.chained_assignment = None
df_list = []
for formula in distri['FORMULA_DISTRI'].unique():
#    print(formula)

    df_new = distri[distri['FORMULA_DISTRI'] == formula].copy()
    df_new['montoDistri'] = pd.eval(formula)
    df_list.append(df_new)
df_distri = pd.concat(df_list, ignore_index=True)
df_distri = df_distri.drop(['IMPUESTO','CPTO_CALCULO'], axis=1)
#df_distri

In [24]:
distri = df_distri.groupby('CTA_DESTINO')['montoDistri'].sum().reset_index()
distri

,CTA_DESTINO,montoDistri
0,900-0000000201/03,156226.120
1,900-0000013093/02,5801.327
2,900-0000251229/08,1822.003
